In [210]:
import os
import shutil
import importlib 

from climatools.cliradlw import (setup, pipeline)
from climatools.lblnew import setup_overlap
from climatools.lblnew import setup_bestfit
import climatools.lblnew.pipeline as pipeline_lblnew
import climatools.lblnew.bestfit_params as bestfit


importlib.reload(setup)
importlib.reload(pipeline)
importlib.reload(setup_overlap)
importlib.reload(setup_bestfit)
importlib.reload(pipeline_lblnew)
importlib.reload(bestfit)

<module 'climatools.lblnew.bestfit_params' from '/chia_cluster/home/jackyu/climatools/climatools/lblnew/bestfit_params.py'>

In [211]:
def molbands():
    return [('h2o', '1'), 
                ('h2o', '2'),
                ('h2o', '3a'),
                ('h2o', '3b'),
                ('h2o', '3c'),
                ('h2o', '4'),
                ('h2o', '5'),
                ('h2o', '6'),
                ('h2o', '7'), 
                ('h2o', '8'),
                ('h2o', '9'),
                ('co2', '3a'), 
                ('co2', '3b'),
                ('co2', '3c'),
                ('co2', '9'),
                ('o3', '5'),
                ('o3', '9'),
                ('n2o', '3a'),
                ('n2o', '7'),
                ('ch4', '7') ]



def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
                     '2': '2',
                     '3a': '3',
                     '3b': '4',
                     '3c': '5',
                     '4': '6', 
                     '5': '7',
                     '6': '8',
                     '7': '9', 
                     '8': '10',
                     '9': '11'}
    
    return lblnew2clirad

In [214]:
def param_lblnew(molecule=None, band=None):
    param = bestfit.kdist_params(molecule=molecule, band=band)
    return param


def param_cliradlw(molecule=None, band=None):
    
    param = bestfit.kdist_params(molecule=molecule, band=band)
    param['atmpro'] = 'mls'

    clirad_band = int(band_map()[band])
    conc = 'atmpro' if param['conc'] == None else param['conc']
    
    return {'atmpro': 'mls',
            'band': [clirad_band],
            'commitnumber': '7a7a0e1',
            'molecule': {molecule: conc},
            'tsfc': param['tsfc']}


def parampairs():
    ps = []
    for molecule, band in molbands():
        ps.append((param_cliradlw(molecule=molecule, band=band), 
                   param_lblnew(molecule=molecule, band=band)))
    return ps


def script():
    ps = parampairs()
    gprocs = pipeline.pipeline_fortran2ipynb2git(parampairs=ps,
                                                 setup=setup)
    
    for param, _ in ps:
        print(pipeline.nbviewer_url(param=param, setup=setup))

In [215]:
script()

Submitting radiation calculation for cases
{'atmpro': 'mls',
 'band': [1],
 'commitnumber': '7a7a0e1',
 'molecule': {'h2o': 'atmpro'},
 'tsfc': 294}
{'atmpro': 'mls',
 'band': [2],
 'commitnumber': '7a7a0e1',
 'molecule': {'h2o': 'atmpro'},
 'tsfc': 294}
{'atmpro': 'mls',
 'band': [3],
 'commitnumber': '7a7a0e1',
 'molecule': {'h2o': 'atmpro'},
 'tsfc': 294}
{'atmpro': 'mls',
 'band': [4],
 'commitnumber': '7a7a0e1',
 'molecule': {'h2o': 'atmpro'},
 'tsfc': 294}
{'atmpro': 'mls',
 'band': [5],
 'commitnumber': '7a7a0e1',
 'molecule': {'h2o': 'atmpro'},
 'tsfc': 294}
{'atmpro': 'mls',
 'band': [6],
 'commitnumber': '7a7a0e1',
 'molecule': {'h2o': 'atmpro'},
 'tsfc': 294}
{'atmpro': 'mls',
 'band': [7],
 'commitnumber': '7a7a0e1',
 'molecule': {'h2o': 'atmpro'},
 'tsfc': 294}
{'atmpro': 'mls',
 'band': [8],
 'commitnumber': '7a7a0e1',
 'molecule': {'h2o': 'atmpro'},
 'tsfc': 294}
{'atmpro': 'mls',
 'band': [9],
 'commitnumber': '7a7a0e1',
 'molecule': {'h2o': 'atmpro'},
 'tsfc': 294}
{'a

{'atmpro': 'mls',
 'band': [6],
 'commitnumber': '7a7a0e1',
 'molecule': {'h2o': 'atmpro'},
 'tsfc': 294}
{'atmpro': 'mls',
 'band': [9],
 'commitnumber': '7a7a0e1',
 'molecule': {'h2o': 'atmpro'},
 'tsfc': 294}
{'atmpro': 'mls',
 'band': [10],
 'commitnumber': '7a7a0e1',
 'molecule': {'h2o': 'atmpro'},
 'tsfc': 294}
{'atmpro': 'mls',
 'band': [11],
 'commitnumber': '7a7a0e1',
 'molecule': {'h2o': 'atmpro'},
 'tsfc': 294}
{'atmpro': 'mls',
 'band': [3],
 'commitnumber': '7a7a0e1',
 'molecule': {'co2': 0.0004},
 'tsfc': 294}
{'atmpro': 'mls',
 'band': [4],
 'commitnumber': '7a7a0e1',
 'molecule': {'co2': 0.0004},
 'tsfc': 294}
{'atmpro': 'mls',
 'band': [5],
 'commitnumber': '7a7a0e1',
 'molecule': {'co2': 0.0004},
 'tsfc': 294}
{'atmpro': 'mls',
 'band': [11],
 'commitnumber': '7a7a0e1',
 'molecule': {'co2': 0.0004},
 'tsfc': 294}
{'atmpro': 'mls',
 'band': [7],
 'commitnumber': '7a7a0e1',
 'molecule': {'o3': 'atmpro'},
 'tsfc': 294}
{'atmpro': 'mls',
 'band': [11],
 'commitnumber': '7